<a href="https://colab.research.google.com/github/mzignis/cats_vs_dogs/blob/master/reused_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
HOME = '/content/drive/My Drive/ml_competition/dogs_vs_cats'
%cd $HOME

/content/drive/My Drive/ml_competition/dogs_vs_cats


In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, optimizers
from keras.applications.vgg16 import VGG16

In [ ]:
sns.set()

In [ ]:
train_dir = os.path.join(HOME, 'data', 'train')
validation_dir = os.path.join(HOME, 'data', 'validation')
test_dir = os.path.join(HOME, 'data', 'test')

## Loading VGG16 model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

### Freezing layers

In [ ]:
base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

### Complie model

In [ ]:
model.compile(loss='binary_crossentropy',
                   optimizer=optimizers.RMSprop(lr=1e-5),
                   metrics=['acc'])

## Preapering data generators

In [ ]:
train_img_gen = ImageDataGenerator(rescale=1./255)
validation_img_gen = ImageDataGenerator(rescale=1./255)
test_img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_img_gen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 20160 images belonging to 2 classes.


In [ ]:
validation_generator = validation_img_gen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 4000 images belonging to 2 classes.


In [ ]:
test_generator = test_img_gen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [ ]:
# history = model.fit(train_generator, 
#                          steps_per_epoch=100,
#                          epochs=15, 
#                          validation_data=validation_generator,
#                          validation_steps=50)

In [ ]:
# model.save('reused_vgg16_model.h5')

In [ ]:
model.load_weights('reused_vgg16_model.h5')

In [ ]:
score, acc = model.evaluate(test_generator)
print('Test score:', score)
print('Test accuracy:', acc)

50/50 [==============================] - 4s 88ms/step - loss: 0.1472 - acc: 0.9370
Test score: 0.14715376496315002
Test accuracy: 0.9369999766349792
